<a href="https://colab.research.google.com/github/olexandryermilov/privacy-preserving/blob/main/deeploma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pip
!pip install transformers[torch]
!pip install sentencepiece
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer

model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-large-ner-hrl")
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-large-ner-hrl")

ner_model = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy = "simple")

     |████████████████████████████████| 1.6 MB 4.1 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 2.9 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 40.7 MB/s 
     |████████████████████████████████| 3.3 MB 46.0 MB/s 
     |████████████████████████████████| 636 kB 43.5 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.7 MB/s 


Downloading:   0%|          | 0.00/982 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/212 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
text = "Hello, Tetyana! My name is Sasha. Sasha is the name given to me by my parents, Misha and Tetyana. I was born in Vinnytsia but now I live in Kyiv. I don't like Kyiv very much."
result = ner_model(text)
print(result)

[{'entity_group': 'PER', 'score': 0.99789494, 'word': 'Tetyana', 'start': 6, 'end': 14}, {'entity_group': 'PER', 'score': 0.99959165, 'word': 'Sasha', 'start': 26, 'end': 32}, {'entity_group': 'PER', 'score': 0.99767387, 'word': 'Sasha', 'start': 33, 'end': 39}, {'entity_group': 'PER', 'score': 0.8411772, 'word': 'Misha', 'start': 78, 'end': 84}, {'entity_group': 'PER', 'score': 0.99283886, 'word': 'Tetyana', 'start': 88, 'end': 96}, {'entity_group': 'LOC', 'score': 0.9995552, 'word': 'Vinnytsia', 'start': 111, 'end': 121}, {'entity_group': 'LOC', 'score': 0.9992939, 'word': 'Kyiv', 'start': 139, 'end': 144}, {'entity_group': 'LOC', 'score': 0.9995993, 'word': 'Kyiv', 'start': 158, 'end': 163}]


In [ ]:
#flow:
#read dataset
#find all NERs 
#Replace NERs with either placeholders or similar entities
#Save dataset
def readFile(filePath):
  return text

def obtain_named_entities(text):
  return ner_model(text)

def replaceNEWithEntityGroup(text, parsed_named_entities):
  anonymizedText = text
  entity_map = dict()
  placeholders_map = dict()
  for entity in reversed(parsed_named_entities):
    start = entity['start']
    end = entity['end']
    entity_word = entity['word']
    if entity_word not in entity_map:
      entity_type = entity['entity_group']
      if entity_type not in placeholders_map:
        placeholders_map[entity_type] = 1
      else:
        placeholders_map[entity_type] = placeholders_map[entity_type] + 1
      entity_map[entity_word] = entity_type + "_" + str(placeholders_map[entity_type])
    replacement = entity_map[entity_word]
    anonymizedText = anonymizedText[:start+1] + replacement + anonymizedText[end:]
  return anonymizedText


def anonymizeCorpus(filePath):
  corpus = readFile(filePath)
  named_entities = obtain_named_entities(corpus)
  anonymized_corpus = replaceNEWithEntityGroup(corpus, named_entities)
  return anonymized_corpus

In [ ]:
anonymizeCorpus('no_file')

"Hello, PER_1! My name is PER_3. PER_3 is the name given to me by my parents, PER_2 and PER_1. I was born in LOC_2 but now I live in LOC_1. I don't like LOC_1 very much"